<a href="https://colab.research.google.com/github/nberonio/Shawnee-Prescribed-Fire/blob/main/FinalHistogramMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00


In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='sixth-firmament-425715-j4')

In [3]:
def lookup(source_hist, target_hist):
    """Creates a lookup table to make a source histogram match a target histogram.

    Args:
        source_hist: The histogram to modify. Expects the Nx2 array format produced by ee.Reducer.autoHistogram.
        target_hist: The histogram to match to. Expects the Nx2 array format produced by ee.Reducer.autoHistogram.

    Returns:
        A dictionary with 'x' and 'y' properties that respectively represent the x and y
        array inputs to the ee.Image.interpolate function.
    """

    # Split the histograms by column and normalize the counts.
    source_values = source_hist.slice(1, 0, 1).project([0])
    source_counts = source_hist.slice(1, 1, 2).project([0])
    source_counts = source_counts.divide(source_counts.get([-1]))

    target_values = target_hist.slice(1, 0, 1).project([0])
    target_counts = target_hist.slice(1, 1, 2).project([0])
    target_counts = target_counts.divide(target_counts.get([-1]))

    # Find first position in target where targetCount >= srcCount[i], for each i.
    def make_lookup(n):
        return target_values.get(target_counts.gte(n).argmax())

    lookup = source_counts.toList().map(make_lookup)

    return {'x': source_values.toList(), 'y': lookup}

In [4]:
def histogram_match(source_img, target_img, geometry):
    """Performs histogram matching for 3-band RGB images by forcing the histogram CDF of source_img to match target_img.

    Args:
        source_img: A 3-band ee.Image to be color matched. Must have bands named 'R', 'G', and 'B'.
        target_img: A 3-band ee.Image for color reference. Must have bands named 'R', 'G', and 'B'.
        geometry: An ee.Geometry that defines the region to generate RGB histograms for.
          It should intersect both source_img and target_img inputs.

    Returns:
        A copy of src_img color-matched to target_img.
    """

    args = {
        'reducer': ee.Reducer.autoHistogram(maxBuckets=256, cumulative=True),
        'geometry': geometry,
        'scale': 1, # Need to specify a scale, but it doesn't matter what it is because bestEffort is true.
        'maxPixels': 65536 * 4 - 1,
        'bestEffort': True
    }

    # Only use pixels in target that have a value in source (inside the footprint and unmasked).
    source = source_img.reduceRegion(**args)
    target = target_img.updateMask(source_img.mask()).reduceRegion(**args)

    return ee.Image(ee.Image.cat(
        source_img.select(['R']).interpolate(**lookup(source.getArray('R'), target.getArray('R'))),
        source_img.select(['G']).interpolate(**lookup(source.getArray('G'), target.getArray('G'))),
        source_img.select(['B']).interpolate(**lookup(source.getArray('B'), target.getArray('B'))),
        source_img.select(['nir']).interpolate(**lookup(source.getArray('nir'), target.getArray('nir')))
    ).copyProperties(source_img, ['system:time_start']))

In [5]:
def find_closest(target_image, image_col, days):
    """Filter images in a collection by date proximity and spatial intersection to a target image.

    Args:
        target_image: An ee.Image whose observation date is used to find near-date images in
          the provided image_col image collection. It must have a 'system:time_start' property.
        image_col: An ee.ImageCollection to filter by date proximity and spatial intersection
          to the target_image. Each image in the collection must have a 'system:time_start'
          property.
        days: A number that defines the maximum number of days difference allowed between
          the target_image and images in the image_col.

    Returns:
        An ee.ImageCollection that has been filtered to include those images that are within the
          given date proximity to target_image and intersect it spatially.
    """

    # Compute the timespan for N days (in milliseconds).
    range = ee.Number(days).multiply(1000 * 60 * 60 * 24)

    filter = ee.Filter.And(
        ee.Filter.maxDifference(range, 'system:time_start', None, 'system:time_start'),
        ee.Filter.intersects('.geo', None, '.geo'))

    closest = (ee.Join.saveAll('matches', 'measure')
        .apply(ee.ImageCollection([target_image]), image_col, filter))

    return ee.ImageCollection(ee.List(closest.first().get('matches')))

In [7]:
feature_collection_Ashby1 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Ashby1_geometry')
geometry_Ashby1 = feature_collection_Ashby1.geometry()
feature_collection_Blowdown = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Blowdown_geometry')
geometry_Blowdown = feature_collection_Blowdown.geometry()
feature_collection_CedarLakeKudzuBurnUnit = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/CedarLakeKudzuBurnUnit_geometry')
geometry_CedarLakeKudzuBurnUnit = feature_collection_CedarLakeKudzuBurnUnit.geometry()
feature_collection_DennisonHollow = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/DennisonHollow_geometry')
geometry_DennisonHollow = feature_collection_DennisonHollow.geometry()
feature_collection_Inagheh1 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Inagheh1_geometry')
geometry_Inagheh1 = feature_collection_Inagheh1.geometry()
feature_collection_JohnsonCo = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/JohnsonCo_geometry')
geometry_JohnsonCo = feature_collection_JohnsonCo.geometry()
feature_collection_LaRue = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/LaRuePineHills_geometry')
geometry_LaRue = feature_collection_LaRue.geometry()
feature_collection_LeeMine = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/LeeMine_geometry')
geometry_LeeMine = feature_collection_LeeMine.geometry()
feature_collection_McConnel1 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/McConnel1_geometry')
geometry_McConnel1 = feature_collection_McConnel1.geometry()
feature_collection_McConnel2 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/McConnel2_geometry')
geometry_McConnel2 = feature_collection_McConnel2.geometry()
feature_collection_McConnel3 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/McConnel3_geometry')
geometry_McConnel3 = feature_collection_McConnel3.geometry()
feature_collection_Oakwood = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Oakwood_geometry')
geometry_Oakwood = feature_collection_Oakwood.geometry()
feature_collection_OzarkHill = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/OzarkHill_geometry')
geometry_OzarkHill = feature_collection_OzarkHill.geometry()
feature_collection_PennantBar1 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/PennantBar1_geometry')
geometry_PennantBar1 = feature_collection_PennantBar1.geometry()
feature_collection_PennantBar2 = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/PennantBar2_geometry')
geometry_PennantBar2 = feature_collection_PennantBar2.geometry()
feature_collection_SimpsonTownshipBarrens_PVT = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/SimpsonTownshipBarrens_PVT_geometry')
geometry_SimpsonTownshipBarrens_PVT = feature_collection_SimpsonTownshipBarrens_PVT.geometry()
feature_collection_TPL = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/TPL_geometry')
geometry_TPL = feature_collection_TPL.geometry()
feature_collection_Turpen = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Turpen_geometry')
geometry_Turpen = feature_collection_Turpen.geometry()
feature_collection_West = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/West_geometry')
geometry_West = feature_collection_West.geometry()
feature_collection_White = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/White_geometry')
geometry_White = feature_collection_White.geometry()
feature_collection_WhoopieCat = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/WhoopieCat_geometry')
geometry_WhoopieCat = feature_collection_WhoopieCat.geometry()
feature_collection_Ashby = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Ashby_geometry')
geometry_Ashby = feature_collection_Ashby.geometry()
feature_collection_Bebout = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Bebout_geometry')
geometry_Bebout = feature_collection_Bebout.geometry()
feature_collection_Inagheh = ee.FeatureCollection('projects/sixth-firmament-425715-j4/assets/Inagheh_geometry')
geometry_Inagheh = feature_collection_Inagheh.geometry()

Ashby1_20180226_Date=ee.Date.fromYMD(2018, 2, 26).millis()
Ashby1_20180302_Date=ee.Date.fromYMD(2018, 3, 2).millis()
Ashby1_20190209_Date=ee.Date.fromYMD(2019, 2, 9).millis()
Blowdown_20210122_Date=ee.Date.fromYMD(2021, 1, 22).millis()
Blowdown_20210205_Date=ee.Date.fromYMD(2021, 2, 5).millis()
Blowdown_20220219_Date=ee.Date.fromYMD(2022, 2, 19 ).millis()
CedarLakeKudzuBurnUnit_20171126_Date=ee.Date.fromYMD(2017, 11, 26).millis()
CedarLakeKudzuBurnUnit_20171206_Date=ee.Date.fromYMD(2017, 12, 6).millis()
CedarLakeKudzuBurnUnit_20181124_Date=ee.Date.fromYMD(2018, 11, 24).millis()
DennisonHollow_20180226_Date=ee.Date.fromYMD(2018, 2, 26).millis()
DennisonHollow_20180306_Date=ee.Date.fromYMD(2018, 3, 6).millis()
DennisonHollow_20190225_Date=ee.Date.fromYMD(2019, 2, 25).millis()
Inagheh1_20211107_Date=ee.Date.fromYMD(2021, 11, 7).millis()
Inagheh1_20211119_Date=ee.Date.fromYMD(2021, 11, 19).millis()
Inagheh1_20221106_Date=ee.Date.fromYMD(2022, 11, 6).millis()
JohnsonCo_20170216_1_Date=ee.Date.fromYMD(2017, 2, 16).millis()
JohnsonCo_20170216_2_Date=ee.Date.fromYMD(2017, 2, 16).millis()
JohnsonCo_20170308_Date=ee.Date.fromYMD(2017, 3, 8).millis()
JohnsonCo_20180208_Date=ee.Date.fromYMD(2018, 2, 8).millis()
LaRue_20170217_Date=ee.Date.fromYMD(2017, 2, 17).millis()
LaRue_20170315_Date=ee.Date.fromYMD(2017, 3, 15).millis()
LaRue_20180306_Date=ee.Date.fromYMD(2018, 3, 6).millis()
LeeMine_20200221_Date=ee.Date.fromYMD(2020, 2, 21).millis()
LeeMine_20200307_Date=ee.Date.fromYMD(2020, 3, 7).millis()
LeeMine_20210304_Date=ee.Date.fromYMD(2021, 3, 4).millis()
McConnel1_20180302_Date=ee.Date.fromYMD(2018, 3, 2).millis()
McConnel1_20180314_Date=ee.Date.fromYMD(2018, 3, 14).millis()
McConnel1_20190316_Date=ee.Date.fromYMD(2019, 3, 16).millis()
McConnel2_20200229_Date=ee.Date.fromYMD(2020, 2, 29).millis()
McConnel2_20200307_Date=ee.Date.fromYMD(2020, 3, 7).millis()
McConnel2_20210303_Date=ee.Date.fromYMD(2021, 3, 3).millis()
McConnel3_20201106_Date=ee.Date.fromYMD(2020, 11, 6).millis()
McConnel3_20201117_Date=ee.Date.fromYMD(2020, 11, 17).millis()
McConnel3_20211116_Date=ee.Date.fromYMD(2021, 11, 16).millis()
Oakwood_20201107_Date=ee.Date.fromYMD(2020, 11, 7).millis()
Oakwood_20201112_Date=ee.Date.fromYMD(2020, 11, 12).millis()
Oakwood_20211107_Date=ee.Date.fromYMD(2021, 11, 7).millis()
OzarkHill_20170217_Date=ee.Date.fromYMD(2017, 2, 17).millis()
OzarkHill_20170407_Date=ee.Date.fromYMD(2017, 4, 7).millis()
OzarkHill_20180227_Date=ee.Date.fromYMD(2018, 2, 27).millis()
PennantBar1_20170205_Date=ee.Date.fromYMD(2017, 2, 5).millis()
PennantBar1_20170216_Date=ee.Date.fromYMD(2017, 2, 16).millis()
PennantBar1_20180208_Date=ee.Date.fromYMD(2018, 2, 8).millis()
PennantBar2_20180304_Date=ee.Date.fromYMD(2018, 3, 4).millis()
PennantBar2_20180314_Date=ee.Date.fromYMD(2018, 3, 14).millis()
PennantBar2_20190316_Date=ee.Date.fromYMD(2019, 3, 16).millis()
SimpsonTownshipBarrens_PVT_20200229_Date=ee.Date.fromYMD(2020, 2, 29).millis()
SimpsonTownshipBarrens_PVT_20200308_Date=ee.Date.fromYMD(2020, 3, 8).millis()
SimpsonTownshipBarrens_PVT_20210304_Date=ee.Date.fromYMD(2021, 3, 4).millis()
TPL_20200119_Date=ee.Date.fromYMD(2020, 1, 19).millis()
TPL_20200227_Date=ee.Date.fromYMD(2020, 2, 27).millis()
TPL_20210205_Date=ee.Date.fromYMD(2021, 2, 5).millis()
Turpen_20161104_Date=ee.Date.fromYMD(2016, 11, 4).millis()
Turpen_20161111_Date=ee.Date.fromYMD(2016, 11, 11).millis()
Turpen_20171109_Date=ee.Date.fromYMD(2017, 11, 9).millis()
West_20171206_Date=ee.Date.fromYMD(2017, 12, 6).millis()
West_20171216_Date=ee.Date.fromYMD(2017, 12, 16).millis()
West_20181211_Date=ee.Date.fromYMD(2018, 12, 11).millis()
White_20180227_Date=ee.Date.fromYMD(2018, 2, 27).millis()
White_20180306_Date=ee.Date.fromYMD(2018, 3, 6).millis()
White_20190322_Date=ee.Date.fromYMD(2019, 3, 22).millis()
WhoopieCat_20200307_Date=ee.Date.fromYMD(2020, 3, 7).millis()
WhoopieCat_20200326_Date=ee.Date.fromYMD(2020, 3, 26).millis()
WhoopieCat_20210307_Date=ee.Date.fromYMD(2021, 3, 7).millis()
Ashby_20190224_Date=ee.Date.fromYMD(2019, 2, 24).millis()
Ashby_20190306_Date=ee.Date.fromYMD(2019, 3, 6).millis()
Ashby_20200222_Date=ee.Date.fromYMD(2020, 2, 22).millis()
Bebout_20210302_Date=ee.Date.fromYMD(2021, 3, 2).millis()
Bebout_20210307_Date=ee.Date.fromYMD(2021, 3, 7).millis()
Bebout_20220303_Date=ee.Date.fromYMD(2022, 3, 3).millis()
Inagheh_20210120_Date=ee.Date.fromYMD(2021, 1, 20).millis()
Inagheh_20210123_Date=ee.Date.fromYMD(2021, 1, 23).millis()
Inagheh_20220125_Date=ee.Date.fromYMD(2022, 1, 25).millis()

Ashby1_20180226 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Ashby1_20180226')
    .clip(geometry_Ashby1).set('system:time_start',Ashby1_20180226_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Ashby1_20180302 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Ashby1_20180302')
    .clip(geometry_Ashby1).set('system:time_start',Ashby1_20180302_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Ashby1_20190209 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Ashby1_20190209')
    .clip(geometry_Ashby1).set('system:time_start',Ashby1_20190209_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Blowdown_20210122 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Blowdown_20210122')
    .clip(geometry_Blowdown).set('system:time_start',Blowdown_20210122_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Blowdown_20210205 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Blowdown_20210205')
    .clip(geometry_Blowdown).set('system:time_start',Blowdown_20210205_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Blowdown_20220219 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Blowdown_20220219')
    .clip(geometry_Blowdown).set('system:time_start',Blowdown_20220219_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
CedarLakeKudzuBurnUnit_20171126 = (ee.Image('projects/sixth-firmament-425715-j4/assets/CedarLakeKudzuBurnUnit_20171126')
    .clip(geometry_CedarLakeKudzuBurnUnit).set('system:time_start',CedarLakeKudzuBurnUnit_20171126_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
CedarLakeKudzuBurnUnit_20171206 = (ee.Image('projects/sixth-firmament-425715-j4/assets/CedarLakeKudzuBurnUnit_20171206')
    .clip(geometry_CedarLakeKudzuBurnUnit).set('system:time_start',CedarLakeKudzuBurnUnit_20171206_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
CedarLakeKudzuBurnUnit_20181124 = (ee.Image('projects/sixth-firmament-425715-j4/assets/CedarLakeKudzuBurnUnit_20181124')
    .clip(geometry_CedarLakeKudzuBurnUnit).set('system:time_start',CedarLakeKudzuBurnUnit_20181124_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
DennisonHollow_20180226 = (ee.Image('projects/sixth-firmament-425715-j4/assets/DennisonHollow_20180226')
    .clip(geometry_DennisonHollow).set('system:time_start',DennisonHollow_20180226_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
DennisonHollow_20180306 = (ee.Image('projects/sixth-firmament-425715-j4/assets/DennisonHollow_20180306')
    .clip(geometry_DennisonHollow).set('system:time_start',DennisonHollow_20180306_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
DennisonHollow_20190225 = (ee.Image('projects/sixth-firmament-425715-j4/assets/DennisonHollow_20190225')
    .clip(geometry_DennisonHollow).set('system:time_start',DennisonHollow_20190225_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Inagheh1_20211107 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Inagheh1_20211107')
    .clip(geometry_Inagheh1).set('system:time_start',Inagheh1_20211107_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Inagheh1_20211119 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Inagheh1_20211119')
    .clip(geometry_Inagheh1).set('system:time_start',Inagheh1_20211119_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Inagheh1_20221106 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Inagheh1_20221106')
    .clip(geometry_Inagheh1).set('system:time_start',Inagheh1_20221106_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
JohnsonCo_20170216_1 = (ee.Image('projects/sixth-firmament-425715-j4/assets/JohnsonCo_20170216_1')
    .clip(geometry_JohnsonCo).set('system:time_start',JohnsonCo_20170216_1_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
JohnsonCo_20170216_2 = (ee.Image('projects/sixth-firmament-425715-j4/assets/JohnsonCo_20170216_2')
    .clip(geometry_JohnsonCo).set('system:time_start',JohnsonCo_20170216_2_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
JohnsonCo_20170308 = (ee.Image('projects/sixth-firmament-425715-j4/assets/JohnsonCo_20170308')
    .clip(geometry_JohnsonCo).set('system:time_start',JohnsonCo_20170308_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
JohnsonCo_20180208 = (ee.Image('projects/sixth-firmament-425715-j4/assets/JohnsonCo_20180208')
    .clip(geometry_JohnsonCo).set('system:time_start',JohnsonCo_20180208_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LaRue_20170217 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LaRue_20170217')
    .clip(geometry_LaRue).set('system:time_start',LaRue_20170217_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LaRue_20170315 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LaRue_20170315')
    .clip(geometry_LaRue).set('system:time_start',LaRue_20170315_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LaRue_20180306 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LaRue_20180306')
    .clip(geometry_LaRue).set('system:time_start',LaRue_20180306_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LeeMine_20200221 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LeeMine_20200221')
    .clip(geometry_LeeMine).set('system:time_start',LeeMine_20200221_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LeeMine_20200307 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LeeMine_20200307')
    .clip(geometry_LeeMine).set('system:time_start',LeeMine_20200307_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
LeeMine_20210304 = (ee.Image('projects/sixth-firmament-425715-j4/assets/LeeMine_20210304')
    .clip(geometry_LeeMine).set('system:time_start',LeeMine_20210304_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel1_20180302 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel1_20180302')
    .clip(geometry_McConnel1).set('system:time_start',McConnel1_20180302_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel1_20180314 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel1_20180314')
    .clip(geometry_McConnel1).set('system:time_start',McConnel1_20180314_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel1_20190316 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel1_20190316')
    .clip(geometry_McConnel1).set('system:time_start',McConnel1_20190316_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel2_20200229 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel2_20200229')
    .clip(geometry_McConnel2).set('system:time_start',McConnel2_20200229_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel2_20200307 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel2_20200307')
    .clip(geometry_McConnel2).set('system:time_start',McConnel2_20200307_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel2_20210303 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel2_20210303')
    .clip(geometry_McConnel2).set('system:time_start',McConnel2_20210303_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel3_20201106 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel3_20201106')
    .clip(geometry_McConnel3).set('system:time_start',McConnel3_20201106_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel3_20201117 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel3_20201117')
    .clip(geometry_McConnel3).set('system:time_start',McConnel3_20201117_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
McConnel3_20211116 = (ee.Image('projects/sixth-firmament-425715-j4/assets/McConnel3_20211116')
    .clip(geometry_McConnel3).set('system:time_start',McConnel3_20211116_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Oakwood_20201107 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Oakwood_20201107')
    .clip(geometry_Oakwood).set('system:time_start',Oakwood_20201107_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Oakwood_20201112 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Oakwood_20201112')
    .clip(geometry_Oakwood).set('system:time_start',Oakwood_20201112_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Oakwood_20211107 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Oakwood_20211107')
    .clip(geometry_Oakwood).set('system:time_start',Oakwood_20211107_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
OzarkHill_20170217 = (ee.Image('projects/sixth-firmament-425715-j4/assets/OzarkHill_20170217')
    .clip(geometry_OzarkHill).set('system:time_start',OzarkHill_20170217_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
OzarkHill_20170407 = (ee.Image('projects/sixth-firmament-425715-j4/assets/OzarkHill_20170407')
    .clip(geometry_OzarkHill).set('system:time_start',OzarkHill_20170407_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
OzarkHill_20180227 = (ee.Image('projects/sixth-firmament-425715-j4/assets/OzarkHill_20180227')
    .clip(geometry_OzarkHill).set('system:time_start',OzarkHill_20180227_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar1_20170205 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar1_20170205')
    .clip(geometry_PennantBar1).set('system:time_start',PennantBar1_20170205_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar1_20170216 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar1_20170216')
    .clip(geometry_PennantBar1).set('system:time_start',PennantBar1_20170216_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar1_20180208 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar1_20180208')
    .clip(geometry_PennantBar1).set('system:time_start',PennantBar1_20180208_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar2_20180304 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar2_20180304')
    .clip(geometry_PennantBar2).set('system:time_start',PennantBar2_20180304_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar2_20180314 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar2_20180314')
    .clip(geometry_PennantBar2).set('system:time_start',PennantBar2_20180314_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
PennantBar2_20190316 = (ee.Image('projects/sixth-firmament-425715-j4/assets/PennantBar2_20190316')
    .clip(geometry_PennantBar2).set('system:time_start',PennantBar2_20190316_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
SimpsonTownshipBarrens_PVT_20200229 = (ee.Image('projects/sixth-firmament-425715-j4/assets/SimpsonTownshipBarrens_PVT_20200229')
    .clip(geometry_SimpsonTownshipBarrens_PVT).set('system:time_start',SimpsonTownshipBarrens_PVT_20200229_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
SimpsonTownshipBarrens_PVT_20200308 = (ee.Image('projects/sixth-firmament-425715-j4/assets/SimpsonTownshipBarrens_PVT_20200308')
    .clip(geometry_SimpsonTownshipBarrens_PVT).set('system:time_start',SimpsonTownshipBarrens_PVT_20200308_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
SimpsonTownshipBarrens_PVT_20210304 = (ee.Image('projects/sixth-firmament-425715-j4/assets/SimpsonTownshipBarrens_PVT_20210304')
    .clip(geometry_SimpsonTownshipBarrens_PVT).set('system:time_start',SimpsonTownshipBarrens_PVT_20210304_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
TPL_20200119 = (ee.Image('projects/sixth-firmament-425715-j4/assets/TPL_20200119')
    .clip(geometry_TPL).set('system:time_start',TPL_20200119_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
TPL_20200227 = (ee.Image('projects/sixth-firmament-425715-j4/assets/TPL_20200227')
    .clip(geometry_TPL).set('system:time_start',TPL_20200227_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
TPL_20210205 = (ee.Image('projects/sixth-firmament-425715-j4/assets/TPL_20210205')
    .clip(geometry_TPL).set('system:time_start',TPL_20210205_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Turpen_20161104 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Turpen_20161104')
    .clip(geometry_Turpen).set('system:time_start',Turpen_20161104_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Turpen_20161111 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Turpen_20161111')
    .clip(geometry_Turpen).set('system:time_start',Turpen_20161111_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Turpen_20171109 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Turpen_20171109')
    .clip(geometry_Turpen).set('system:time_start',Turpen_20171109_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
West_20171206 = (ee.Image('projects/sixth-firmament-425715-j4/assets/West_20171206')
    .clip(geometry_West).set('system:time_start',West_20171206_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
West_20171216 = (ee.Image('projects/sixth-firmament-425715-j4/assets/West_20171216')
    .clip(geometry_West).set('system:time_start',West_20171216_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
West_20181211 = (ee.Image('projects/sixth-firmament-425715-j4/assets/West_20181211')
    .clip(geometry_West).set('system:time_start',West_20181211_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
White_20180227 = (ee.Image('projects/sixth-firmament-425715-j4/assets/White_20180227')
    .clip(geometry_White).set('system:time_start',White_20180227_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
White_20180306 = (ee.Image('projects/sixth-firmament-425715-j4/assets/White_20180306')
    .clip(geometry_White).set('system:time_start',White_20180306_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
White_20190322 = (ee.Image('projects/sixth-firmament-425715-j4/assets/White_20190322')
    .clip(geometry_White).set('system:time_start',White_20190322_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
WhoopieCat_20200307 = (ee.Image('projects/sixth-firmament-425715-j4/assets/WhoopieCat_20200307')
    .clip(geometry_WhoopieCat).set('system:time_start',WhoopieCat_20200307_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
WhoopieCat_20200326 = (ee.Image('projects/sixth-firmament-425715-j4/assets/WhoopieCat_20200326')
    .clip(geometry_WhoopieCat).set('system:time_start',WhoopieCat_20200326_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
WhoopieCat_20210307 = (ee.Image('projects/sixth-firmament-425715-j4/assets/WhoopieCat_20210307')
    .clip(geometry_WhoopieCat).set('system:time_start',WhoopieCat_20210307_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Ashby_20190224 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Ashby_20190224')
    .clip(geometry_Ashby).set('system:time_start',Ashby_20190224_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Ashby_20190306 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Ashby_20190306')
    .clip(geometry_Ashby).set('system:time_start',Ashby_20190306_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Ashby_20200222 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Ashby_20200222')
    .clip(geometry_Ashby).set('system:time_start',Ashby_20200222_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Bebout_20210302 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Bebout_20210302')
    .clip(geometry_Bebout).set('system:time_start',Bebout_20210302_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Bebout_20210307 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Bebout_20210307')
    .clip(geometry_Bebout).set('system:time_start',Bebout_20210307_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Bebout_20220303 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Bebout_20220303')
    .clip(geometry_Bebout).set('system:time_start',Bebout_20220303_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Inagheh_20210120 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Inagheh_01202021')
    .clip(geometry_Inagheh).set('system:time_start',Inagheh_20210120_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Inagheh_20210123 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Inagheh_01232021')
    .clip(geometry_Inagheh).set('system:time_start',Inagheh_20210123_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))
Inagheh_20220125 = (ee.Image('projects/sixth-firmament-425715-j4/assets/Inagheh_01252022')
    .clip(geometry_Inagheh).set('system:time_start',Inagheh_20220125_Date)
    .select(['b3', 'b2', 'b1', 'b4'], ['R', 'G', 'B', 'nir']))

In [8]:
def prep_HLS(image):
    """Scale, apply cloud/shadow mask, and select/rename Landsat 8 bands."""
    qa_mask = image.select('Fmask').bitwiseAnd(int('11111', 2)).eq(0)

    def get_factor_img(factor_names):
        factor_list = image.toDictionary().select(factor_names).values()
        return ee.Image.constant(factor_list)

    scaled = image.multiply(1)

    return image.addBands(scaled, None, True).select(
        ['B4', 'B3', 'B2', 'B5'], ['R', 'G', 'B', 'nir']).updateMask(qa_mask)

# Get the HLS collection, cloud masked and scaled to surface reflectance.
HLS_col_Ashby1 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Ashby1).map(prep_HLS)
HLS_col_Blowdown = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Blowdown).map(prep_HLS)
HLS_col_Inagheh1 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Inagheh1).map(prep_HLS)
HLS_col_CedarLakeKudzuBurnUnit = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_CedarLakeKudzuBurnUnit).map(prep_HLS)
HLS_col_DennisonHollow = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_DennisonHollow).map(prep_HLS)
HLS_col_JohnsonCo = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_JohnsonCo).map(prep_HLS)
HLS_col_LaRue = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_LaRue).map(prep_HLS)
HLS_col_LeeMine = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_LeeMine).map(prep_HLS)
HLS_col_McConnel1 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_McConnel1).map(prep_HLS)
HLS_col_McConnel2 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_McConnel2).map(prep_HLS)
HLS_col_McConnel3 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_McConnel3).map(prep_HLS)
HLS_col_Oakwood = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Oakwood).map(prep_HLS)
HLS_col_OzarkHill = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_OzarkHill).map(prep_HLS)
HLS_col_PennantBar1 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_PennantBar1).map(prep_HLS)
HLS_col_PennantBar2 = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_PennantBar2).map(prep_HLS)
HLS_col_SimpsonTownshipBarrens_PVT = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_SimpsonTownshipBarrens_PVT).map(prep_HLS)
HLS_col_TPL = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_TPL).map(prep_HLS)
HLS_col_Turpen = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Turpen).map(prep_HLS)
HLS_col_West = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_West).map(prep_HLS)
HLS_col_White = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_White).map(prep_HLS)
HLS_col_WhoopieCat = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_WhoopieCat).map(prep_HLS)
HLS_col_Ashby = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Ashby).map(prep_HLS)
HLS_col_Bebout = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Bebout).map(prep_HLS)
HLS_col_Inagheh = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterBounds(
    geometry_Inagheh).map(prep_HLS)

In [9]:
reference_Ashby1_20180226 = find_closest(Ashby1_20180226, HLS_col_Ashby1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Ashby1_20180302 = find_closest(Ashby1_20180302, HLS_col_Ashby1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Ashby1_20190209 = find_closest(Ashby1_20190209, HLS_col_Ashby1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Blowdown_20210122 = find_closest(Blowdown_20210122, HLS_col_Blowdown, 50).sort('CLOUD_COVERAGE').mosaic()
reference_Blowdown_20210205 = find_closest(Blowdown_20210205, HLS_col_Blowdown, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Blowdown_20220219 = find_closest(Blowdown_20220219, HLS_col_Blowdown, 50).sort('CLOUD_COVERAGE').mosaic()
reference_CedarLakeKudzuBurnUnit_20171126 = find_closest(CedarLakeKudzuBurnUnit_20171126, HLS_col_CedarLakeKudzuBurnUnit, 32).sort('CLOUD_COVERAGE').mosaic()
reference_CedarLakeKudzuBurnUnit_20171206 = find_closest(CedarLakeKudzuBurnUnit_20171206, HLS_col_CedarLakeKudzuBurnUnit, 32).sort('CLOUD_COVERAGE').mosaic()
reference_CedarLakeKudzuBurnUnit_20181124 = find_closest(CedarLakeKudzuBurnUnit_20181124, HLS_col_CedarLakeKudzuBurnUnit, 32).sort('CLOUD_COVERAGE').mosaic()
reference_DennisonHollow_20180226 = find_closest(DennisonHollow_20180226, HLS_col_DennisonHollow, 32).sort('CLOUD_COVERAGE').mosaic()
reference_DennisonHollow_20180306 = find_closest(DennisonHollow_20180306, HLS_col_DennisonHollow, 32).sort('CLOUD_COVERAGE').mosaic()
reference_DennisonHollow_20190225 = find_closest(DennisonHollow_20190225, HLS_col_DennisonHollow, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Inagheh1_20211107 = find_closest(Inagheh1_20211107, HLS_col_Inagheh1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Inagheh1_20211119 = find_closest(Inagheh1_20211119, HLS_col_Inagheh1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Inagheh1_20221106 = find_closest(Inagheh1_20221106, HLS_col_Inagheh1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_JohnsonCo_20170216_1 = find_closest(JohnsonCo_20170216_1, HLS_col_JohnsonCo, 32).sort('CLOUD_COVERAGE').mosaic()
reference_JohnsonCo_20170216_2 = find_closest(JohnsonCo_20170216_2, HLS_col_JohnsonCo, 32).sort('CLOUD_COVERAGE').mosaic()
reference_JohnsonCo_20170308 = find_closest(JohnsonCo_20170308, HLS_col_JohnsonCo, 32).sort('CLOUD_COVERAGE').mosaic()
reference_JohnsonCo_20180208 = find_closest(JohnsonCo_20180208, HLS_col_JohnsonCo, 32).sort('CLOUD_COVERAGE').mosaic()
reference_LaRue_20170217 = find_closest(LaRue_20170217, HLS_col_LaRue, 32).sort('CLOUD_COVERAGE').mosaic()
reference_LaRue_20170315 = find_closest(LaRue_20170315, HLS_col_LaRue, 32).sort('CLOUD_COVERAGE').mosaic()
reference_LaRue_20180306 = find_closest(LaRue_20180306, HLS_col_LaRue, 64).sort('CLOUD_COVERAGE').mosaic()
reference_LeeMine_20200221 = find_closest(LeeMine_20200221, HLS_col_LeeMine, 32).sort('CLOUD_COVERAGE').mosaic()
reference_LeeMine_20200307 = find_closest(LeeMine_20200307, HLS_col_LeeMine, 32).sort('CLOUD_COVERAGE').mosaic()
reference_LeeMine_20210304 = find_closest(LeeMine_20210304, HLS_col_LeeMine, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel1_20180302 = find_closest(McConnel1_20180302, HLS_col_McConnel1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel1_20180314 = find_closest(McConnel1_20180314, HLS_col_McConnel1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel1_20190316 = find_closest(McConnel1_20190316, HLS_col_McConnel1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel2_20200229 = find_closest(McConnel2_20200229, HLS_col_McConnel2, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel2_20200307 = find_closest(McConnel2_20200307, HLS_col_McConnel2, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel2_20210303 = find_closest(McConnel2_20210303, HLS_col_McConnel2, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel3_20201106 = find_closest(McConnel3_20201106, HLS_col_McConnel3, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel3_20201117 = find_closest(McConnel3_20201117, HLS_col_McConnel3, 32).sort('CLOUD_COVERAGE').mosaic()
reference_McConnel3_20211116 = find_closest(McConnel3_20211116, HLS_col_McConnel3, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Oakwood_20201107 = find_closest(Oakwood_20201107, HLS_col_Oakwood, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Oakwood_20201112 = find_closest(Oakwood_20201112, HLS_col_Oakwood, 64).sort('CLOUD_COVERAGE').mosaic()
reference_Oakwood_20211107 = find_closest(Oakwood_20211107, HLS_col_Oakwood, 32).sort('CLOUD_COVERAGE').mosaic()
reference_OzarkHill_20170217 = find_closest(OzarkHill_20170217, HLS_col_OzarkHill, 32).sort('CLOUD_COVERAGE').mosaic()
reference_OzarkHill_20170407 = find_closest(OzarkHill_20170407, HLS_col_OzarkHill, 32).sort('CLOUD_COVERAGE').mosaic()
reference_OzarkHill_20180227 = find_closest(OzarkHill_20180227, HLS_col_OzarkHill, 32).sort('CLOUD_COVERAGE').mosaic()
reference_PennantBar1_20170205 = find_closest(PennantBar1_20170205, HLS_col_PennantBar1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_PennantBar1_20170216 = find_closest(PennantBar1_20170216, HLS_col_PennantBar1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_PennantBar1_20180208 = find_closest(PennantBar1_20180208, HLS_col_PennantBar1, 32).sort('CLOUD_COVERAGE').mosaic()
reference_PennantBar2_20180304 = find_closest(PennantBar2_20180304, HLS_col_PennantBar2, 32).sort('CLOUD_COVERAGE').mosaic()
reference_PennantBar2_20180314 = find_closest(PennantBar2_20180314, HLS_col_PennantBar2, 32).sort('CLOUD_COVERAGE').mosaic()
reference_PennantBar2_20190316 = find_closest(PennantBar2_20190316, HLS_col_PennantBar2, 32).sort('CLOUD_COVERAGE').mosaic()
reference_SimpsonTownshipBarrens_PVT_20200229 = find_closest(SimpsonTownshipBarrens_PVT_20200229, HLS_col_SimpsonTownshipBarrens_PVT, 32).sort('CLOUD_COVERAGE').mosaic()
reference_SimpsonTownshipBarrens_PVT_20200308 = find_closest(SimpsonTownshipBarrens_PVT_20200308, HLS_col_SimpsonTownshipBarrens_PVT, 32).sort('CLOUD_COVERAGE').mosaic()
reference_SimpsonTownshipBarrens_PVT_20210304 = find_closest(SimpsonTownshipBarrens_PVT_20210304, HLS_col_SimpsonTownshipBarrens_PVT, 32).sort('CLOUD_COVERAGE').mosaic()
reference_TPL_20200119 = find_closest(TPL_20200119, HLS_col_TPL, 32).sort('CLOUD_COVERAGE').mosaic()
reference_TPL_20200227 = find_closest(TPL_20200227, HLS_col_TPL, 32).sort('CLOUD_COVERAGE').mosaic()
reference_TPL_20210205 = find_closest(TPL_20210205, HLS_col_TPL, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Turpen_20161104 = find_closest(Turpen_20161104, HLS_col_Turpen, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Turpen_20161111 = find_closest(Turpen_20161111, HLS_col_Turpen, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Turpen_20171109 = find_closest(Turpen_20171109, HLS_col_Turpen, 32).sort('CLOUD_COVERAGE').mosaic()
reference_West_20171206 = find_closest(West_20171206, HLS_col_West, 32).sort('CLOUD_COVERAGE').mosaic()
reference_West_20171216 = find_closest(West_20171216, HLS_col_West, 32).sort('CLOUD_COVERAGE').mosaic()
reference_West_20181211 = find_closest(West_20181211, HLS_col_West, 32).sort('CLOUD_COVERAGE').mosaic()
reference_White_20180227 = find_closest(White_20180227, HLS_col_White, 32).sort('CLOUD_COVERAGE').mosaic()
reference_White_20180306 = find_closest(White_20180306, HLS_col_White, 64).sort('CLOUD_COVERAGE').mosaic()
reference_White_20190322 = find_closest(White_20190322, HLS_col_White, 50).sort('CLOUD_COVERAGE').mosaic()
reference_WhoopieCat_20200307 = find_closest(WhoopieCat_20200307, HLS_col_WhoopieCat, 32).sort('CLOUD_COVERAGE').mosaic()
reference_WhoopieCat_20200326 = find_closest(WhoopieCat_20200326, HLS_col_WhoopieCat, 32).sort('CLOUD_COVERAGE').mosaic()
reference_WhoopieCat_20210307 = find_closest(WhoopieCat_20210307, HLS_col_WhoopieCat, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Ashby_20190224 = find_closest(Ashby_20190224, HLS_col_Ashby, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Ashby_20190306 = find_closest(Ashby_20190306, HLS_col_Ashby, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Ashby_20200222 = find_closest(Ashby_20200222, HLS_col_Ashby, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Bebout_20210302 = find_closest(Bebout_20210302, HLS_col_Bebout, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Bebout_20210307 = find_closest(Bebout_20210307, HLS_col_Bebout, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Bebout_20220303 = find_closest(Bebout_20220303, HLS_col_Bebout, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Inagheh_20210120 = find_closest(Inagheh_20210120, HLS_col_Inagheh, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Inagheh_20210123 = find_closest(Inagheh_20210123, HLS_col_Inagheh, 32).sort('CLOUD_COVERAGE').mosaic()
reference_Inagheh_20220125 = find_closest(Inagheh_20220125, HLS_col_Inagheh, 32).sort('CLOUD_COVERAGE').mosaic()

In [10]:
result_Ashby1_20180226 = histogram_match(Ashby1_20180226, reference_Ashby1_20180226, geometry_Ashby1)
result_Ashby1_20180302 = histogram_match(Ashby1_20180302, reference_Ashby1_20180302, geometry_Ashby1)
result_Ashby1_20190209 = histogram_match(Ashby1_20190209, reference_Ashby1_20190209, geometry_Ashby1)
result_Blowdown_20210122 = histogram_match(Blowdown_20210122, reference_Blowdown_20210122, geometry_Blowdown)
result_Blowdown_20210205 = histogram_match(Blowdown_20210205, reference_Blowdown_20210205, geometry_Blowdown)
result_Blowdown_20220219 = histogram_match(Blowdown_20220219, reference_Blowdown_20220219, geometry_Blowdown)
result_CedarLakeKudzuBurnUnit_20171126 = histogram_match(CedarLakeKudzuBurnUnit_20171126, reference_CedarLakeKudzuBurnUnit_20171126, geometry_CedarLakeKudzuBurnUnit)
result_CedarLakeKudzuBurnUnit_20171206 = histogram_match(CedarLakeKudzuBurnUnit_20171206, reference_CedarLakeKudzuBurnUnit_20171206, geometry_CedarLakeKudzuBurnUnit)
result_CedarLakeKudzuBurnUnit_20181124 = histogram_match(CedarLakeKudzuBurnUnit_20181124, reference_CedarLakeKudzuBurnUnit_20181124, geometry_CedarLakeKudzuBurnUnit)
result_DennisonHollow_20180226 = histogram_match(DennisonHollow_20180226, reference_DennisonHollow_20180226, geometry_DennisonHollow)
result_DennisonHollow_20180306 = histogram_match(DennisonHollow_20180306, reference_DennisonHollow_20180306, geometry_DennisonHollow)
result_DennisonHollow_20190225 = histogram_match(DennisonHollow_20190225, reference_DennisonHollow_20190225, geometry_DennisonHollow)
result_Inagheh1_20211107 = histogram_match(Inagheh1_20211107, reference_Inagheh1_20211107, geometry_Inagheh1)
result_Inagheh1_20211119 = histogram_match(Inagheh1_20211119, reference_Inagheh1_20211119, geometry_Inagheh1)
result_Inagheh1_20221106 = histogram_match(Inagheh1_20221106, reference_Inagheh1_20221106, geometry_Inagheh1)
result_JohnsonCo_20170216_1 = histogram_match(JohnsonCo_20170216_1, reference_JohnsonCo_20170216_1, geometry_JohnsonCo)
result_JohnsonCo_20170216_2 = histogram_match(JohnsonCo_20170216_2, reference_JohnsonCo_20170216_2, geometry_JohnsonCo)
result_JohnsonCo_20170308 = histogram_match(JohnsonCo_20170308, reference_JohnsonCo_20170308, geometry_JohnsonCo)
result_JohnsonCo_20180208 = histogram_match(JohnsonCo_20180208, reference_JohnsonCo_20180208, geometry_JohnsonCo)
result_LaRue_20170217 = histogram_match(LaRue_20170217, reference_LaRue_20170217, geometry_LaRue)
result_LaRue_20170315 = histogram_match(LaRue_20170315, reference_LaRue_20170315, geometry_LaRue)
result_LaRue_20180306 = histogram_match(LaRue_20180306, reference_LaRue_20180306, geometry_LaRue)
result_LeeMine_20200221 = histogram_match(LeeMine_20200221, reference_LeeMine_20200221, geometry_LeeMine)
result_LeeMine_20200307 = histogram_match(LeeMine_20200307, reference_LeeMine_20200307, geometry_LeeMine)
result_LeeMine_20210304 = histogram_match(LeeMine_20210304, reference_LeeMine_20210304, geometry_LeeMine)
result_McConnel1_20180302 = histogram_match(McConnel1_20180302, reference_McConnel1_20180302, geometry_McConnel1)
result_McConnel1_20180314 = histogram_match(McConnel1_20180314, reference_McConnel1_20180314, geometry_McConnel1)
result_McConnel1_20190316 = histogram_match(McConnel1_20190316, reference_McConnel1_20190316, geometry_McConnel1)
result_McConnel2_20200229 = histogram_match(McConnel2_20200229, reference_McConnel2_20200229, geometry_McConnel2)
result_McConnel2_20200307 = histogram_match(McConnel2_20200307, reference_McConnel2_20200307, geometry_McConnel2)
result_McConnel2_20210303 = histogram_match(McConnel2_20210303, reference_McConnel2_20210303, geometry_McConnel2)
result_McConnel3_20201106 = histogram_match(McConnel3_20201106, reference_McConnel3_20201106, geometry_McConnel3)
result_McConnel3_20201117 = histogram_match(McConnel3_20201117, reference_McConnel3_20201117, geometry_McConnel3)
result_McConnel3_20211116 = histogram_match(McConnel3_20211116, reference_McConnel3_20211116, geometry_McConnel3)
result_Oakwood_20201107 = histogram_match(Oakwood_20201107, reference_Oakwood_20201107, geometry_Oakwood)
result_Oakwood_20201112 = histogram_match(Oakwood_20201112, reference_Oakwood_20201112, geometry_Oakwood)
result_Oakwood_20211107 = histogram_match(Oakwood_20211107, reference_Oakwood_20211107, geometry_Oakwood)
result_OzarkHill_20170217 = histogram_match(OzarkHill_20170217, reference_OzarkHill_20170217, geometry_OzarkHill)
result_OzarkHill_20170407 = histogram_match(OzarkHill_20170407, reference_OzarkHill_20170407, geometry_OzarkHill)
result_OzarkHill_20180227 = histogram_match(OzarkHill_20180227, reference_OzarkHill_20180227, geometry_OzarkHill)
result_PennantBar1_20170205 = histogram_match(PennantBar1_20170205, reference_PennantBar1_20170205, geometry_PennantBar1)
result_PennantBar1_20170216 = histogram_match(PennantBar1_20170216, reference_PennantBar1_20170216, geometry_PennantBar1)
result_PennantBar1_20180208 = histogram_match(PennantBar1_20180208, reference_PennantBar1_20180208, geometry_PennantBar1)
result_PennantBar2_20180304 = histogram_match(PennantBar2_20180304, reference_PennantBar2_20180304, geometry_PennantBar2)
result_PennantBar2_20180314 = histogram_match(PennantBar2_20180314, reference_PennantBar2_20180314, geometry_PennantBar2)
result_PennantBar2_20190316 = histogram_match(PennantBar2_20190316, reference_PennantBar2_20190316, geometry_PennantBar2)
result_SimpsonTownshipBarrens_PVT_20200229 = histogram_match(SimpsonTownshipBarrens_PVT_20200229, reference_SimpsonTownshipBarrens_PVT_20200229, geometry_SimpsonTownshipBarrens_PVT)
result_SimpsonTownshipBarrens_PVT_20200308 = histogram_match(SimpsonTownshipBarrens_PVT_20200308, reference_SimpsonTownshipBarrens_PVT_20200308, geometry_SimpsonTownshipBarrens_PVT)
result_SimpsonTownshipBarrens_PVT_20210304 = histogram_match(SimpsonTownshipBarrens_PVT_20210304, reference_SimpsonTownshipBarrens_PVT_20210304, geometry_SimpsonTownshipBarrens_PVT)
result_TPL_20200119 = histogram_match(TPL_20200119, reference_TPL_20200119, geometry_TPL)
result_TPL_20200227 = histogram_match(TPL_20200227, reference_TPL_20200227, geometry_TPL)
result_TPL_20210205 = histogram_match(TPL_20210205, reference_TPL_20210205, geometry_TPL)
result_Turpen_20161104 = histogram_match(Turpen_20161104, reference_Turpen_20161104, geometry_Turpen)
result_Turpen_20161111 = histogram_match(Turpen_20161111, reference_Turpen_20161111, geometry_Turpen)
result_Turpen_20171109 = histogram_match(Turpen_20171109, reference_Turpen_20171109, geometry_Turpen)
result_West_20171206 = histogram_match(West_20171206, reference_West_20171206, geometry_West)
result_West_20171216 = histogram_match(West_20171216, reference_West_20171216, geometry_West)
result_West_20181211 = histogram_match(West_20181211, reference_West_20181211, geometry_West)
result_White_20180227 = histogram_match(White_20180227, reference_White_20180227, geometry_White)
result_White_20180306 = histogram_match(White_20180306, reference_White_20180306, geometry_White)
result_White_20190322 = histogram_match(White_20190322, reference_White_20190322, geometry_White)
result_WhoopieCat_20200307 = histogram_match(WhoopieCat_20200307, reference_WhoopieCat_20200307, geometry_WhoopieCat)
result_WhoopieCat_20200326 = histogram_match(WhoopieCat_20200326, reference_WhoopieCat_20200326, geometry_WhoopieCat)
result_WhoopieCat_20210307 = histogram_match(WhoopieCat_20210307, reference_WhoopieCat_20210307, geometry_WhoopieCat)
result_Ashby_20190224 = histogram_match(Ashby_20190224, reference_Ashby_20190224, geometry_Ashby)
result_Ashby_20190306 = histogram_match(Ashby_20190306, reference_Ashby_20190306, geometry_Ashby)
result_Ashby_20200222 = histogram_match(Ashby_20200222, reference_Ashby_20200222, geometry_Ashby)
result_Bebout_20210302 = histogram_match(Bebout_20210302, reference_Bebout_20210302, geometry_Bebout)
result_Bebout_20210307 = histogram_match(Bebout_20210307, reference_Bebout_20210307, geometry_Bebout)
result_Bebout_20220303 = histogram_match(Bebout_20220303, reference_Bebout_20220303, geometry_Bebout)
result_Inagheh_20210120 = histogram_match(Inagheh_20210120, reference_Inagheh_20210120, geometry_Inagheh)
result_Inagheh_20210123 = histogram_match(Inagheh_20210123, reference_Inagheh_20210123, geometry_Inagheh)
result_Inagheh_20220125 = histogram_match(Inagheh_20220125, reference_Inagheh_20220125, geometry_Inagheh)

In [11]:
task_config_Ashby1 = {'scale': 3,'region': geometry_Ashby1, 'maxPixels': 1e13}
task_config_Blowdown = {'scale': 3,'region': geometry_Blowdown, 'maxPixels': 1e13}
task_config_CedarLakeKudzuBurnUnit = {'scale': 3,'region': geometry_CedarLakeKudzuBurnUnit, 'maxPixels': 1e13}
task_config_DennisonHollow = {'scale': 3,'region': geometry_DennisonHollow, 'maxPixels': 1e13}
task_config_Inagheh1 = {'scale': 3,'region': geometry_Inagheh1, 'maxPixels': 1e13}
task_config_JohnsonCo = {'scale': 3,'region': geometry_JohnsonCo, 'maxPixels': 1e13}
task_config_LaRue = {'scale': 3,'region': geometry_LaRue, 'maxPixels': 1e13}
task_config_LeeMine = {'scale': 3,'region': geometry_LeeMine, 'maxPixels': 1e13}
task_config_McConnel1 = {'scale': 3,'region': geometry_McConnel1, 'maxPixels': 1e13}
task_config_McConnel2 = {'scale': 3,'region': geometry_McConnel2, 'maxPixels': 1e13}
task_config_McConnel3 = {'scale': 3,'region': geometry_McConnel3, 'maxPixels': 1e13}
task_config_Oakwood = {'scale': 3,'region': geometry_Oakwood, 'maxPixels': 1e13}
task_config_OzarkHill = {'scale': 3,'region': geometry_OzarkHill, 'maxPixels': 1e13}
task_config_PennantBar1 = {'scale': 3,'region': geometry_PennantBar1, 'maxPixels': 1e13}
task_config_PennantBar2 = {'scale': 3,'region': geometry_PennantBar2, 'maxPixels': 1e13}
task_config_SimpsonTownshipBarrens_PVT = {'scale': 3,'region': geometry_SimpsonTownshipBarrens_PVT, 'maxPixels': 1e13}
task_config_TPL = {'scale': 3,'region': geometry_TPL, 'maxPixels': 1e13}
task_config_Turpen = {'scale': 3,'region': geometry_Turpen, 'maxPixels': 1e13}
task_config_West = {'scale': 3,'region': geometry_West, 'maxPixels': 1e13}
task_config_White = {'scale': 3,'region': geometry_White, 'maxPixels': 1e13}
task_config_WhoopieCat = {'scale': 3,'region': geometry_WhoopieCat, 'maxPixels': 1e13}
task_config_Ashby = {'scale': 3,'region': geometry_Ashby, 'maxPixels': 1e13}
task_config_Bebout = {'scale': 3,'region': geometry_Bebout, 'maxPixels': 1e13}
task_config_Inagheh = {'scale': 3,'region': geometry_Inagheh, 'maxPixels': 1e13}

task1 = ee.batch.Export.image(result_Ashby1_20180226, 'Ashby1_20180226', task_config_Ashby1)
task2 = ee.batch.Export.image(result_Ashby1_20180302, 'Ashby1_20180302', task_config_Ashby1)
task3 = ee.batch.Export.image(result_Ashby1_20190209, 'Ashby1_20190209', task_config_Ashby1)
task4 = ee.batch.Export.image(result_Blowdown_20210122, 'Blowdown_20210122', task_config_Blowdown)
task5 = ee.batch.Export.image(result_Blowdown_20210205, 'Blowdown_20210203', task_config_Blowdown)
task6 = ee.batch.Export.image(result_Blowdown_20220219, 'Blowdown_20220219', task_config_Blowdown)
task7 = ee.batch.Export.image(result_CedarLakeKudzuBurnUnit_20171126, 'CedarLakeKudzuBurnUnit_20171126', task_config_CedarLakeKudzuBurnUnit)
task8 = ee.batch.Export.image(result_CedarLakeKudzuBurnUnit_20171206, 'CedarLakeKudzuBurnUnit_20171206', task_config_CedarLakeKudzuBurnUnit)
task9 = ee.batch.Export.image(result_CedarLakeKudzuBurnUnit_20181124, 'CedarLakeKudzuBurnUnit_20181124', task_config_CedarLakeKudzuBurnUnit)
task10 = ee.batch.Export.image(result_DennisonHollow_20180226, 'DennisonHollow_20180226', task_config_DennisonHollow)
task11 = ee.batch.Export.image(result_DennisonHollow_20180306, 'DennisonHollow_20180306', task_config_DennisonHollow)
task12 = ee.batch.Export.image(result_DennisonHollow_20190225, 'DennisonHollow_20190225', task_config_DennisonHollow)
task13 = ee.batch.Export.image(result_Inagheh1_20211107, 'Inagheh1_20211107', task_config_Inagheh)
task14 = ee.batch.Export.image(result_Inagheh1_20211119, 'Inagheh1_20211119', task_config_Inagheh)
task15 = ee.batch.Export.image(result_Inagheh1_20221106, 'Inagheh1_20221106', task_config_Inagheh)
task16 = ee.batch.Export.image(result_JohnsonCo_20170216_1, 'JohnsonCo_20170216_1', task_config_JohnsonCo)
task16_2 = ee.batch.Export.image(result_JohnsonCo_20170216_2, 'JohnsonCo_20170216_2', task_config_JohnsonCo)
task17 = ee.batch.Export.image(result_JohnsonCo_20170308, 'JohnsonCo_20170308', task_config_JohnsonCo)
task18 = ee.batch.Export.image(result_JohnsonCo_20180208, 'JohnsonCo_20180208', task_config_JohnsonCo)
task19 = ee.batch.Export.image(result_LaRue_20170217, 'LaRue_20170217', task_config_LaRue)
task20 = ee.batch.Export.image(result_LaRue_20170315, 'LaRue_20170315', task_config_LaRue)
task21 = ee.batch.Export.image(result_LaRue_20180306, 'LaRue_20180306', task_config_LaRue)
task22 = ee.batch.Export.image(result_LeeMine_20200221, 'LeeMine_20200221', task_config_LeeMine)
task23 = ee.batch.Export.image(result_LeeMine_20200307, 'LeeMine_20200307', task_config_LeeMine)
task24 = ee.batch.Export.image(result_LeeMine_20210304, 'LeeMine_20210304', task_config_LeeMine)
task25 = ee.batch.Export.image(result_McConnel1_20180302, 'McConnel1_20180302', task_config_McConnel1)
task26 = ee.batch.Export.image(result_McConnel1_20180314, 'McConnel1_20180314', task_config_McConnel1)
task27 = ee.batch.Export.image(result_McConnel1_20190316, 'McConnel1_20190316', task_config_McConnel1)
task28 = ee.batch.Export.image(result_McConnel2_20200229, 'McConnel2_20200229', task_config_McConnel2)
task29 = ee.batch.Export.image(result_McConnel2_20200307, 'McConnel2_20200307', task_config_McConnel2)
task30 = ee.batch.Export.image(result_McConnel2_20210303, 'McConnel2_20210303', task_config_McConnel2)
task31 = ee.batch.Export.image(result_McConnel3_20201106, 'McConnel3_20201106', task_config_McConnel3)
task32 = ee.batch.Export.image(result_McConnel3_20201117, 'McConnel3_20201117', task_config_McConnel3)
task33 = ee.batch.Export.image(result_McConnel3_20211116, 'McConnel3_20211116', task_config_McConnel3)
task34 = ee.batch.Export.image(result_Oakwood_20201107, 'Oakwood_20201107', task_config_Oakwood)
task35 = ee.batch.Export.image(result_Oakwood_20201112, 'Oakwood_20201112', task_config_Oakwood)
task36 = ee.batch.Export.image(result_Oakwood_20211107, 'Oakwood_20211107', task_config_Oakwood)
task37 = ee.batch.Export.image(result_OzarkHill_20170217, 'OzarkHill_20170217', task_config_OzarkHill)
task38 = ee.batch.Export.image(result_OzarkHill_20170407, 'OzarkHill_20170407', task_config_OzarkHill)
task39 = ee.batch.Export.image(result_OzarkHill_20180227, 'OzarkHill_20180227', task_config_OzarkHill)
task40 = ee.batch.Export.image(result_PennantBar1_20170205, 'PennantBar1_20170205', task_config_PennantBar1)
task41 = ee.batch.Export.image(result_PennantBar1_20170216, 'PennantBar1_20170216', task_config_PennantBar1)
task42 = ee.batch.Export.image(result_PennantBar1_20180208, 'PennantBar1_20180208', task_config_PennantBar1)
task43 = ee.batch.Export.image(result_PennantBar2_20180304, 'PennantBar2_20180304', task_config_PennantBar2)
task44 = ee.batch.Export.image(result_PennantBar2_20180314, 'PennantBar2_20180314', task_config_PennantBar2)
task45 = ee.batch.Export.image(result_PennantBar2_20190316, 'PennantBar2_20190316', task_config_PennantBar2)
task46 = ee.batch.Export.image(result_SimpsonTownshipBarrens_PVT_20200229, 'SimpsonTownshipBarrens_PVT_20200229', task_config_SimpsonTownshipBarrens_PVT)
task47 = ee.batch.Export.image(result_SimpsonTownshipBarrens_PVT_20200308, 'SimpsonTownshipBarrens_PVT_20200308', task_config_SimpsonTownshipBarrens_PVT)
task48 = ee.batch.Export.image(result_SimpsonTownshipBarrens_PVT_20210304, 'SimpsonTownshipBarrens_PVT_20210304', task_config_SimpsonTownshipBarrens_PVT)
task49 = ee.batch.Export.image(result_TPL_20200119, 'TPL_20200119', task_config_TPL)
task50 = ee.batch.Export.image(result_TPL_20200227, 'TPL_20200227', task_config_TPL)
task51 = ee.batch.Export.image(result_TPL_20210205, 'TPL_20210205', task_config_TPL)
task52 = ee.batch.Export.image(result_Turpen_20161104, 'Turpen_20161104', task_config_Turpen)
task53 = ee.batch.Export.image(result_Turpen_20161111, 'Turpen_20161111', task_config_Turpen)
task54 = ee.batch.Export.image(result_Turpen_20171109, 'Turpen_20171109', task_config_Turpen)
task55 = ee.batch.Export.image(result_West_20171206, 'West_20171206', task_config_West)
task56 = ee.batch.Export.image(result_West_20171216, 'West_20171216', task_config_West)
task57 = ee.batch.Export.image(result_West_20181211, 'West_20181211', task_config_West)
task58 = ee.batch.Export.image(result_White_20180227, 'White_20180227', task_config_White)
task59 = ee.batch.Export.image(result_White_20180306, 'White_20180306', task_config_White)
task60 = ee.batch.Export.image(result_White_20190322, 'White_20190322', task_config_White)
task61 = ee.batch.Export.image(result_WhoopieCat_20200307, 'WhoopieCat_20200307', task_config_WhoopieCat)
task62 = ee.batch.Export.image(result_WhoopieCat_20200326, 'WhoopieCat_20200326', task_config_WhoopieCat)
task63 = ee.batch.Export.image(result_WhoopieCat_20210307, 'WhoopieCat_20210307', task_config_WhoopieCat)
task64 = ee.batch.Export.image(result_Ashby_20190224, 'Ashby_20190224', task_config_Ashby)
task65 = ee.batch.Export.image(result_Ashby_20190306, 'Ashby_20190306', task_config_Ashby)
task66 = ee.batch.Export.image(result_Ashby_20200222, 'Ashby_20200222', task_config_Ashby)
task67 = ee.batch.Export.image(result_Bebout_20210302, 'Bebout_20210302', task_config_Bebout)
task68 = ee.batch.Export.image(result_Bebout_20210307, 'Bebout_20210307', task_config_Bebout)
task69 = ee.batch.Export.image(result_Bebout_20220303, 'Bebout_20220303', task_config_Bebout)
task70 = ee.batch.Export.image(result_Inagheh_20210120, 'Inagheh_20210120', task_config_Inagheh)
task71 = ee.batch.Export.image(result_Inagheh_20210123, 'Inagheh_20210123', task_config_Inagheh)
task72 = ee.batch.Export.image(result_Inagheh_20220125, 'Inagheh_20220125', task_config_Inagheh)

task1.start()
task2.start()
task3.start()
task4.start()
task5.start()
task6.start()
task7.start()
task8.start()
task9.start()
task10.start()
task11.start()
task12.start()
task13.start()
task14.start()
task15.start()
task16.start()
task16_2.start()
task17.start()
task18.start()
task19.start()
task20.start()
task21.start()
task22.start()
task23.start()
task24.start()
task25.start()
task26.start()
task27.start()
task28.start()
task29.start()
task30.start()
task31.start()
task32.start()
task33.start()
task34.start()
task35.start()
task36.start()
task37.start()
task38.start()
task39.start()
task40.start()
task41.start()
task42.start()
task43.start()
task44.start()
task45.start()
task46.start()
task47.start()
task48.start()
task49.start()
task50.start()
task51.start()
task52.start()
task53.start()
task54.start()
task55.start()
task56.start()
task57.start()
task58.start()
task59.start()
task60.start()
task61.start()
task62.start()
task63.start()
task64.start()
task65.start()
task66.start()
task67.start()
task68.start()
task69.start()
task70.start()
task71.start()
task72.start()

'\ntask1.start()\ntask2.start()\ntask3.start()\ntask4.start()\ntask5.start()\ntask6.start()\ntask7.start()\ntask8.start()\ntask9.start()\ntask10.start()\ntask11.start()\ntask12.start()\ntask13.start()\ntask14.start()\ntask15.start()\ntask16.start()\ntask16_2.start()\ntask17.start()\ntask18.start()\ntask19.start()\ntask20.start()\ntask21.start()\ntask22.start()\ntask23.start()\ntask24.start()\ntask25.start()\ntask26.start()\ntask27.start()\ntask28.start()\ntask29.start()\ntask30.start()\ntask31.start()\ntask32.start()\ntask33.start()\ntask34.start()\ntask35.start()\ntask36.start()\ntask37.start()\ntask38.start()\ntask39.start()\ntask40.start()\ntask41.start()\ntask42.start()\ntask43.start()\ntask44.start()\ntask45.start()\ntask46.start()\ntask47.start()\ntask48.start()\ntask49.start()\ntask50.start()\ntask51.start()\ntask52.start()\ntask53.start()\ntask54.start()\ntask55.start()\ntask56.start()\ntask57.start()\ntask58.start()\ntask59.start()\ntask60.start()\ntask61.start()\ntask62.start